# **Deep Learning per rilevare la Polmonite da immagini a raggi X**

Questo algoritmo identifica automaticamente se un paziente soffre o meno di polmonite osservando le radiografie del torace. Visto che sono in gioco le vite delle persone, questo algoritmo deve essere estremamente accurato.



---



## Importazione moduli

In [ ]:
# Librerie generiche
import os             # Si importa la libreria
import numpy as np    # Si importa la libreria assegnandole il nome 'np'
import pandas as pd   # Si importa la libreria assegnandole il nome 'pd'
import random         # Si importa la libreria 
import cv2            # Si importa la libreria
import matplotlib.pyplot as plt   # Si importa una funzione della libreria assegnandole il nome 'plt'
!pip install requests
import requests

# !pip install requests
# from urllib.request import urlretrieve
import urllib.request


# Il comando seguente è un comando di shell che fa sì che i dati dei grafici siano mostrati nel Jupiter Notebook (Google Colaboratory lo è) (viene usato perché se no i grafici sarebbero delle singole finestre, che non sarebbe possibile mostrare in un notebook o da browser)
%matplotlib inline

# Librerie per il machine learning
import keras.backend as K   # Si importa una funzione della sottolibreria backend di keras (backend è una sottolibreria di keras) assegnandole il nome 'K'
from keras.models import Model, Sequential    # Si importano due funzioni della della sottolibreria models di keras
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization   # Si importano cinque funzioni della sottolibreria layers di keras
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation    # Si importano altre cinque funzioni della sottolibreria layers di keras
from keras.optimizer_v2 import adam
from keras.preprocessing.image import ImageDataGenerator  # Si importa una funzione della sottolibreria image della sottolibreria preprocessing di keras
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping   # Si importano tre funzioni della sottolibreria callbacks di keras
import tensorflow as tf   # Si importa la libreria assegnandole il nome 'tf'

# Variabili per rendere riproducibile l'esperimento (parametri che rendono la casualità uguale per tutti)
seed = 232    # Assegnazione di un valore ad una variabile
np.random.seed(seed)    # Inserimento della variabile in una funzione come inizializzazione della libreria NumPy (np)
tf.random.set_seed(seed)    # Inserimento della variabile in una funzione come inizializzazione della libreria TensorFlow (tf)

*LIBRERIE*
*   **OS**: Os serve per interagire con il sistema operativo (Operating System).
*   **NumPy**: Numpy serve per l'analisi numerica (NumericalPython).
*   **Pandas**: Pandas serve per manipolare e analizzare i dati.
*   **Random**: Random serve per generare dei numeri semi-casuali (infatti per un dispositivo deterministico come un computer o algoritmo è impossibile generare valori randomici visto che darà ogni volta lo stesso output da una data condizione di partenza o stato iniziale, definiti nella sezione di definizione della variabile 'seed').
*   **OpenCV (cv2)**: OpenCV serve per la computer vision.
*   **Matplotlib**: Matplotlib serve per la creazione di grafici.
*   **Keras**: Keras serve da interfaccia per il Machine Learning e le reti neurali (solitamente fornite da TensorFlow).
*   **TensorFlow**: TensorFlow serve per sviluppare e addestrare modelli di Machine Learning.









## Montaggio del drive per recuperare le immagini di addestramento, di test e di validazione

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Assegnazione ad una variabile il percorso della cartella con tutte le immagini di addestramento, di test e di validazione

È necessario caricare su drive la cartella con le immagini di addestramento, test e validazione dal sito https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia. Inserire poi il percorso della cartella nella variabile 'input_path'.

In [ ]:
input_path = '/content/drive/MyDrive/chest_xray/'

## Visualizzazione di immagini di esempio prese dai tre dataset di allenamento, validazione e test con e senza polmonite

In [ ]:
# Mostra 3 paia di immagini prese dai tre dataset di allenamento, validazione e test senza e con polmonite
fig, ax = plt.subplots(2, 3, figsize=(15, 7))
ax = ax.ravel()
plt.tight_layout()

for i, _set in enumerate(['train', 'val', 'test']):
    set_path = input_path+_set
    ax[i].imshow(plt.imread(set_path+'/NORMAL/'+os.listdir(set_path+'/NORMAL')[0]), cmap='gray')
    ax[i].set_title('Set: {} - Condizione: Sano'.format(_set))
    ax[i+3].imshow(plt.imread(set_path+'/PNEUMONIA/'+os.listdir(set_path+'/PNEUMONIA')[0]), cmap='gray')
    ax[i+3].set_title('Set: {} - Condizione: Polmonite'.format(_set))

## Definizioni per l'addestramento

In [ ]:
# Distribuzione dei dataset
for _set in ['train', 'val', 'test']:
    n_normal = len(os.listdir(input_path + _set + '/NORMAL'))
    n_infect = len(os.listdir(input_path + _set + '/PNEUMONIA'))
    print('Set: {}, immagini sane: {}, immagini polmoniti: {}'.format(_set, n_normal, n_infect))

In [ ]:
def process_data(img_dims, batch_size):
    # Definizione dei dati delle generazioni di allenamento e di test
    train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, vertical_flip=True)
    test_val_datagen = ImageDataGenerator(rescale=1./255)
    
    # Definizione delle generazioni di allenamento e di test
    train_gen = train_datagen.flow_from_directory(
    directory=input_path+'train', 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='binary', 
    shuffle=True)

    test_gen = test_val_datagen.flow_from_directory(
    directory=input_path+'test', 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='binary', 
    shuffle=True)
    
    # Creazione di previsioni del set di test, utile per avere una matrice di confusione/confusion matrix (tabella usata per valutare le performance di un algoritmo)
    test_data = []
    test_labels = []

    for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (os.listdir(input_path + 'test' + cond)):
            img = plt.imread(input_path+'test'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/NORMAL/':
                label = 0
            elif cond=='/PNEUMONIA/':
                label = 1
            test_data.append(img)
            test_labels.append(label)
        
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    
    return train_gen, test_gen, test_data, test_labels

In [ ]:
# Variabili utilizzate per l'addestramento della macchina
img_dims = 150
epochs = 2
batch_size = 32

# Elaborazione dei dati
train_gen, test_gen, test_data, test_labels = process_data(img_dims, batch_size)

CONVOLUZIONE: operazione tra due funzioni che restituisce una terza funzione che esprime come la forma di una è modificata dall'altra funzione iniziale.

In [ ]:
# Livello di input
inputs = Input(shape=(img_dims, img_dims, 3))

# Primo blocco di convoluzione
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Secondo blocco di convoluzione
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Terzo blocco di convoluzione
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Quarto blocco di convoluzione
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Quinto blocco di convoluzione
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Livello FC (Fully Connected, ovvero che collega tutti i neuroni di un livello con quelli di un'altro)
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(rate=0.3)(x)

# Livello di output
output = Dense(units=1, activation='sigmoid')(x)

# Creazione del modello e compilazione
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks (funzioni che consentono di avere una visuale del modello durante l'allenamento)
checkpoint = ModelCheckpoint(filepath='best_weights.hdf5', save_best_only=True, save_weights_only=True)   # Salvataggio del modello dopo ogni epoca
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')    # Interrompe l'allenamento se il modello smette di migliorare
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')   # Interrompe l'addestramento in caso di sovradattamento (quando la funzione impara troppo bene solo i valori di addestramento)

## Addestramento del modello

In [ ]:
# Addestramento del modello
hist = model.fit_generator(
           train_gen, steps_per_epoch=train_gen.samples // batch_size, 
           epochs=epochs, validation_data=test_gen, 
           validation_steps=test_gen.samples // batch_size, callbacks=[checkpoint, lr_reduce])

## Visualizzazione dei grafici di accuratezza e perdita

In [ ]:
# Mostra grafici di accuratezza e perdita
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['accuracy', 'loss']):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history['val_' + met])
    ax[i].set_title('Modello {}'.format(met))
    ax[i].set_xlabel('epoche')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

## Test del modello allenato con immagini di test che l'algoritmo non ha ancora visto

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

preds = model.predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

# Stampa dei risultati dei parametri
print('MATRICE DI CONFUSIONE ------------------')
print(cm)
print('\nPARAMETRI DI TEST ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
print('Accuratezza: {}%'.format(acc))
print('Precisione: {}%'.format(precision))
print('Sensibilità: {}%'.format(recall))    # Proporzione dei veri positivi
print('F1-score: {}'.format(2*precision*recall/(precision+recall)))   # F1-score è una media armonica di precisione e sensibilità
print('\PARAMETRI DI ADDESTRAMENTO ----------------------')
print('Accuratezza modello addestrato: {}'.format(np.round((hist.history['accuracy'][-1])*100, 2)))